<a href="https://colab.research.google.com/github/lucas-t-reis/CUDA/blob/master/notebooks/Professioanl_CUDA_programming_exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- Checking for Nvidia **CUDA toolkit**
- Checking for Nvidia Devices (enabled in Runtime type)

In [0]:
!which nvcc
!ls -l /dev/nv*

/usr/local/cuda/bin/nvcc
crw-rw-rw- 1 root root 195,   0 Jan  7 21:21 /dev/nvidia0
crw-rw-rw- 1 root root 195, 255 Jan  7 21:21 /dev/nvidiactl
crw-rw-rw- 1 root root 247,   0 Jan  7 21:21 /dev/nvidia-uvm
crw-rw-rw- 1 root root 247,   1 Jan  7 21:21 /dev/nvidia-uvm-tools


### Installing plugin to allow running CUDA C/C++ - by [andreinechaev](https://github.com/andreinechaev)

In [3]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-nddf24pt
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-nddf24pt
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4307 sha256=ff2c6f9347660956ae9b8e1f975f8ff938e44bbdb54b556fc0d11b73dff0f809
  Stored in directory: /tmp/pip-ephem-wheel-cache-ustdcvfk/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


# Hello World - Example 1

In [11]:
%%cu
#include <stdio.h>

// __global__ qualifier to denote a function called by CPU and executed by GPU
__global__ void helloWorldGPU() {
    
    int gpu_tid = threadIdx.x;
    printf("Hello World from GPU thread %d!\n", gpu_tid);
}

int main() {
    printf("Hello World from CPU!\n");
    
    // Kernel
    helloWorldGPU <<< 1,10 >>> ();

    cudaDeviceReset();
}

Hello World from CPU!
Hello World from GPU thread 0!
Hello World from GPU thread 1!
Hello World from GPU thread 2!
Hello World from GPU thread 3!
Hello World from GPU thread 4!
Hello World from GPU thread 5!
Hello World from GPU thread 6!
Hello World from GPU thread 7!
Hello World from GPU thread 8!
Hello World from GPU thread 9!

